In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from sklearn import svm
import matplotlib.pyplot as plt

- Datasets to use: 
    - https://ballotpedia.org/State_legislative_elections,_2022
        labeled: chamberDf -> senateDf, houseDf
    - https://www.gunviolencearchive.org/reports/mass-shooting?year=2022
        labeled: shootingsDf
     - https://www.rand.org/pubs/tools/TLA243-2-v2.html
         labeled: lawsDf

In [ ]:
shootingsDf = pd.read_csv('/kaggle/input/gunviolencearchive2022/export-f85f25fd-88b3-470c-8c6a-8282d7aba586.csv')

In [ ]:
city_col = shootingsDf['City Or County']
shootingsDf.drop(columns=['City Or County'], inplace=True)
shootingsDf.insert(0, 'City Or County', city_col)

state_col = shootingsDf['State']
shootingsDf.drop(columns=['State'], inplace=True)
shootingsDf.insert(0, 'State', state_col)
shootingsDf = shootingsDf.drop('Operations', axis=1)
shootingsDf

In [ ]:
#add a casualities column to have both killed and injured as total
shootingsDf['Casualties']=shootingsDf['# Killed']+shootingsDf['# Injured']
shootingsDf.head

In [ ]:
shootingsDf['Incident Date'] = pd.to_datetime(shootingsDf['Incident Date'])

# Create the line plot using seaborn
sns.lineplot(x='Incident Date', y='Casualties', data=shootingsDf)

# Set the plot title and axis labels
plt.title('Number of Casualties by Date')
plt.xlabel('Date')
plt.ylabel('Number of Casualties')

# Display the plot
plt.show()

In [ ]:
url = 'https://ballotpedia.org/State_legislative_elections,_2022'
response = requests.get(url)
# parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

# find the tbody containing the data
table = soup.find_all('tbody')[5]
rows = table.find_all('tr')
data = []
for row in rows:
    cols = row.find_all('td')
    cols = [col.get_text().strip() for col in cols]
    #print(cols)
    data.append(cols)
del data[0:2]
chamberDf = pd.DataFrame(data, columns=['Chamber', 'Primary Date', 'Majority'])
chamberDf['Majority'] = chamberDf['Majority'].replace(['Power-sharing agreement'],'Split')

In [ ]:
def get_state_name(chamber_name):
    state_name = chamber_name.split()[0]
    if state_name == 'New' or state_name == 'North' or state_name == 'South' or state_name == 'West':
        state_name += ' ' + chamber_name.split()[1]
    return state_name
chamberDf['State'] = chamberDf['Chamber'].apply(get_state_name)

In [ ]:
chamberDf = chamberDf.reset_index(drop=True)
last_col = chamberDf.iloc[:, -1]  # select the last column
chamberDf = chamberDf.iloc[:, :-1]  # drop the last column
chamberDf.insert(0, 'State', last_col) #add it as the first hehe

In [ ]:
#spliitng chambers df into senate and others
mask = chamberDf.Chamber.str.contains("Senate")
senateDf = chamberDf[mask]
houseDf = chamberDf[~mask]
senateDf.set_axis(['state', 'chamber', 'senPrimaryDate', 'senateMajority'], axis=1, inplace=False)
houseDf.set_axis(['state', 'chamber', 'housePrimaryDate', 'houseMajority'],axis=1, inplace=False)
senateDf.reset_index()
houseDf.reset_index()

In [ ]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
          'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
          'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
          'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
          'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
          'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',
          'Wisconsin', 'Wyoming']

df = pd.DataFrame(states, columns=['State'])

In [ ]:
senateDf = pd.merge(senateDf, df, on='State', how='outer')
houseDf = pd.merge(houseDf, df, on='State', how='outer')
senateDf = senateDf.rename(columns={'Majority':'senateMajority'})
houseDf = houseDf.rename(columns={'Majority':'houseMajority'})

In [ ]:
senateDf = senateDf.drop('Chamber', axis=1)
senateDf = senateDf.drop('Primary Date', axis=1)
houseDf = houseDf.drop('Chamber', axis=1)
houseDf = houseDf.drop('Primary Date', axis=1)

Analysis:

In [ ]:
# manually fill in the bottom values
senateDf.loc[43:50, 'senateMajority'] = ['Republican', 'Republican', 'Democratic', 'Democratic', 'Democratic', 'Democratic', 'Republican', 'Republican']


In [ ]:
senateDf = senateDf.drop(senateDf[senateDf['State'] == 'Rhode'].index)


In [ ]:
senateDf

In [ ]:
# manually fill in the bottom values
houseDf.loc[45:50, 'houseMajority'] = ['Republican', 'Democratic', 'Republican', 'Democratic', 'Democratic','Democratic']

In [ ]:
houseDf = houseDf.drop(houseDf[houseDf['State'] == 'Rhode'].index)

In [ ]:
houseDf

In [ ]:
#group by
shootingsDf_gb = shootingsDf.groupby('State')
shootingsDf_gb['Casualties'].agg( ['count', 'min', 'max', 'mean', 'median', 'std', 'sum'] )

In [ ]:
shootings_by_state = shootingsDf['State'].value_counts()
shootingsDf_percentage = (shootings_by_state / shootings_by_state.sum()) * 100

In [ ]:
plt.figure(figsize=(8, 6))
shootingsDf_percentage.sort_values(ascending=True).plot(kind='barh', color='blue')
plt.title('Percentage of Mass Shootings by State')
plt.xlabel('Percentage')
plt.ylabel('State')
plt.show()

In [ ]:
senateMerged = pd.merge(shootingsDf, senateDf, on = 'State',how='left')
senateMerged

In [ ]:
houseMerged = pd.merge(shootingsDf, houseDf, on = 'State',how='left')
houseMerged

Now we have merged dataframes with the house and senate along side every instance of a mass shooting. 